# Modeling anabolic pathways

Building and simulation pathways using COBRApy. 


In [ ]:
import cobra
import escher
from qbio_resources.histidine_pathway import metabolites_and_info, reactions_and_info

In [ ]:
# The reactions and metabolites to be added in this exercise
builder = escher.Builder(map_json='../qbio_resources/histidine_ribose_map.json')
builder.display_in_notebook()

---------
## A) Add the pathway metabolites and their information to a new model

The goal of this exercise will be to build a simulate the histidine synthesis pathway shown above. 

This section will introduce the first steps of adding the pathway to model.

### 1) Initialize an empty COBRApy model

### 2) Add metabolites and info to the model

**Hints:**

1) The information regarding the reactions and metabolites involved in this pathway can be found in `reactions_and_info` and `metabolites_and_info`, respectively. These are imported above.
  - These are imported in the form of a **dictionary**

In [ ]:
print(metabolites_and_info)

----

- A dictionary is a Python data structure that consists of both `keys` and `values`. This allows you to look up quantities based on the `key` you are interested in. An example of this is shown below for the histidine metabolite

----

In [ ]:
# Look up the information assocated with the histidine metabolite (his__L_c)
print('Values associated with his__L_c = ', metabolites_and_info['his__L_c'])
# This returns another dictionary where the formula, etc. can be looked up with:
print('Formula = ', metabolites_and_info['his__L_c']['formula'])

2) To leverage dictionary properties to add the metabolites to model more efficiently, you can iterate through all of the dictionary entries (items), like below

In [ ]:
for key, value in metabolites_and_info.items():
    print(key, value)

-------
## B) Add reactions to model and simulate

### 1) Like above, add the reaction info found in `reactions_and_info` to the model
 

### 2)  Mass balance check each reaction added to the pathway using `reaction.check_mass_balance()` method

This is an important step in quality controlling any metabolic model. Failing to do so could result in incorrect reactions and incorrect predictions

**Hints:** 

1) An example of using this function can be found below


In [ ]:
# Returning an empty dictionary means the reaction is mass balanced
model.reactions.HISTD.check_mass_balance()

2) You can iterate through all reactions in a model with the following

In [ ]:
for r in model.reactions:
    print(r.id, r.reaction)

### 3) If a reaction is not mass balanced, correct it with `reaction.add_metabolites()` and confirm it is now mass balance
The proper stoichiometry for the reaction can be found at [ecocyc](http://www.ecocyc.com) or [bigg](http://bigg.ucsd.edu). Databases like these are useful references for metabolic reconstructions and understanding models.

**Hint:**
1) Metabolites can be add and removed from reactions using `add_metabolites` on an existing reaction like below

In [ ]:
model.reactions.ATPPRT.add_metabolites({'his__L_c': 1}, combine=False)
print('With histidine: ', model.reactions.ATPPRT.reaction)
model.reactions.ATPPRT.add_metabolites({'his__L_c': 0}, combine=False)
print('Back to original reaction: ', model.reactions.ATPPRT.reaction)

### 4) Add boundary reactions to act as source and sink for ribose and histidine

Models require inputs and outputs in order to simulate. In other words, flux going into a metabolite must ultimately go somewhere.

The reactions should have the following info:

| ID      | Reaction       |   Lower Bound | Upper Bound 
| :-------------: |:-------------:|:-------------:|:-------------:|
| EX_rib__D_e | rib__D_e <=>     | -10 | 1000 |
| EX_his__L_e | his__L_e -->     | 0 | 1000 |

These reactions are called **Exchange Reactions** and by convention begin with 'EX_'. Further, by convention a negative lower bound/flux corresponds to a model input (or metabolite uptake from the growth media/growth condition). Therefore, a lower bound of 10 means that the model can take up, at most, 10 $\frac{mmol}{gDW \cdot hr}$ of ribose

### 5) View the growth media of the model based on the exchange reactions using `model.medium`

### 6) Set the model objective to histidine production ('EX_his__L_e') and optimize

**Hint:** The objective corresponds to the reaction flux that the optimization will either maximize or minimize (by default maximize). It can be changed with the following:

In [ ]:
model.objective = 'HISTD'
print(model.objective)

**Still no flux through the reaction?  Why?**

Flux balance analysis assumes that the system is opperating at steady state (meaning the concentration of the metabolites in the system do not increase or decrease over time). As a consequence of this, each metabolite that participates in the pathway must be created and consumed at the same rate. 

-----
## C) Simulate pathway using complete model
The above exercise demonstrates that a more complete network is required to model the activity of the histidine synthesis pathway. For this section, we will try to model the pathway activity using the most complete metabolic reconstruction of *E. coli*, iML1515.

### 1) Load the model and assign it to variable called `full_model`
The model path is '../qbio_resources/iML1515.json'

In [ ]:
# Set the lower bound of ATPM reaction to 0. This will be discussed later
full_model.reactions.ATPM.lower_bound = 0

### 2) Change the primary carbon substrate from glucose (`EX_glc__D_e`) to ribose (`EX_rib__D_e`)
Allow a maximum of 10 $\frac{mmol}{gDW \cdot hr}$ of uptake flux

### 3) Change the model objective to histidine secretion (`EX_his__L_e`)

### 4) Optimize the model using parsimonious FBA (pFBA)
This is an optimization method that finds the most frugal way to optimize a given objective. In other words, it gives the optimal solution that requires the least amount of total reaction flux.

Information about how the function can be used can by found using:

In [ ]:
cobra.flux_analysis.pfba?

The `fraction_of_optimum`, `objective`, and `reactions` parameters can be left as their default. 

Assign the output of this function to a variable named `solution`

### 5) Visualize the flux solution on an escher map using the `visualize_flux()` function below

**Hint:**  The reaction fluxes can be accessed from the `solution` object returned from the pfba function with `solution.fluxes`

In [ ]:
def visualize_flux(reaction_fluxes):
    builder = escher.Builder(map_json='../qbio_resources/primer_iML_map.json', 
                         reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'value', 'value': .01, 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#0000dd', 'size': 20}])
    builder.reaction_data = reaction_fluxes.to_dict()
    
    return builder.display_in_notebook()

### 6) Optimize the model anaerobically and visualize the solution on an escher map

The default *in silico* media is aerobic glucose M9 minimal media

The id for oxygen uptake is `EX_o2_e`

In [ ]:
# Below list the rest of the metabolites in the default growth media. 
full_model.medium

### 7) (If time permits) Find the number of reactions required to synthesize histidine from ribose, anaerobically

**Hint:** the reaction flux from the most recent optimization can be found using the following:

In [ ]:
full_model.reactions.EX_his__L_e.flux